## Least-Square Fitting with [`SciPy`](https://docs.scipy.org/doc/scipy/reference/)

In [ ]:
from scipy.optimize import least_squares
from scipy.optimize import curve_fit
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize': [8, 5], 'font.size':16, 'legend.fontsize':14, 
                     'legend.handlelength':1, 'legend.frameon':False,
                     'xtick.major.size':8,'ytick.major.size':8,
                     'xtick.minor.size':4,'ytick.minor.size':4,
                     'xtick.direction':'in','ytick.direction':'in',
                     'xtick.labelsize':14,'ytick.labelsize':14,
                     'xtick.major.pad':5,'ytick.major.pad':5,
                     'axes.linewidth':1.5,'axes.labelsize':16,})

### Case Study: Time-Resolved Fluorescence Spectroscopy

<table><tr><td><img src="https://marcello-sega.github.io/pytim/_images/micelle2.png" width="200"><figcaption>A micelle</figcaption></td><td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Pyrene.svg/190px-Pyrene.svg.png" width="170"><figcaption>Pyrene</figcaption></td><td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Benzophenon.svg/1200px-Benzophenon.svg.png" width="190"><figcaption>Benzophenone</figcaption></td></tr><table>

### SDS Micelles in the Presence of a Hydrophobic Fluorophore
Sodium dodecyl sulfate (SDS) is an anionic surfactant which self-assembles into micelles in aqueous solutions.
At the critical micellar concentration (C.M.C.), surfactant molecules form micelles with a well-defined aggregation number. Above the C.M.C., the number of micelles increases while the size of the micelles stays the same.<br>
Pyrene is a hydrophobic fluorescent molecule. In an aqueous SDS solution at [SDS]>C.M.C., pyrene partitions into the hydrophobic interior of the micelles.<br>
The fluorescence intensity decay of pyrene following an excitation impulse is given by<br>
$F(t)=F(0)exp(-t/\tau_0)$<br>
where $\tau_0$ is the lifetime of the fluorophore.
Fluorescence counts follows Poisson's statistics, therefore the error in $F(t)$ is simply $\sqrt{F(t)}$.

### Fluorescence Quenching by Benzophenone
Benzophenone is a hydrophobic compound which quenches the fluorescence of pyrene in the micelles. 
The fluorescence intensity decay of pyrene in the presence of benzophenone is given by<br>
$F(t)=F(0)exp\{-t/\tau_0-m[1-exp(-k_qt)]\}$<br>
where $m$ is average number of quenchers per micelle while $k_q$ is the quenching constant.<br>
Here, we perform a global fit on the fluorescence intensity decays of pyrene in SDS dispersions containing increasing concentrations of benzophenone.<br>
We obtain the fitting parameters $F(0)$, $\tau_0$, $m$, and $k_q$.

The data are tab-separated values (TSV) files (collected from [Prof. L. Stella Lab](http://www.stc.uniroma2.it/physchem/stella.html))

In [ ]:
!head -n 13 'data/fluorescence/quencher_0.0.txt'

In [ ]:
!ls 'data/fluorescence'

In [ ]:
for conc in [0,0.2,0.5,1.0,1.5]:
    t,F = np.loadtxt('data/fluorescence/quencher_{:1.1f}.txt'.format(conc), unpack=True, skiprows=9)
    plt.errorbar(t, F, np.sqrt(F), lw=0, marker='o', label='{:1.1f}'.format(conc)+' mM',
                 ms=4, elinewidth=1., capsize=3, capthick=1., mfc='w')
plt.legend(title='[quencher]',labelspacing=.7)
plt.ylabel('$F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

In [ ]:
for conc in [0,0.2,0.5,1.0,1.5]:
    d = np.loadtxt('data/fluorescence/quencher_{:1.1f}.txt'.format(conc), unpack=False, skiprows=9)
    d = d[ d[:,0] > d[d[:,1]==d[:,1].max(), 0] ] # points before the intesity max are discarded
    plt.errorbar(d[:,0]-d[0,0], d[:,1], np.sqrt(d[:,1]), lw=0, marker='o', label='{:1.1f}'.format(conc)+' mM',
                 ms=4, elinewidth=1., capsize=3, capthick=1., mfc='w')
plt.legend(title='[quencher]',labelspacing=.7)
plt.ylabel('$F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

#### A convenient way to save data sets to file is to use [pickling](https://docs.python.org/3/library/pickle.html). We create a  `pandas` `DataFrame` and we save it as a binary file.

In [ ]:
# first we generate a dictionary
data = {}
for conc in [0,0.2,0.5,1.0,1.5]:
    d = np.loadtxt('data/fluorescence/quencher_{:1.1f}.txt'.format(conc), unpack=False, skiprows=9)
    d = d[ d[:,0] > d[d[:,1]==d[:,1].max(), 0] ] # points before the intesity max are discarded
    data[conc] = { 't': d[:,0]-d[0,0], 'F': d[:,1], 'Ferr': np.sqrt(d[:,1]) }
# then a dataframe is generated from the dictionary
import pandas as pd
dataframe = pd.DataFrame(data)
dataframe.to_pickle('data/fluorescence.p')
dataframe

#### The curve for [quencher]=0 mM is fitted to a single exponential decay using [`optimize.least_square`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html)
`
least_squares(function_to_minimize, initial_params, 
    [x, y, yerr], bounds=[(lower_bounds),(upper_bounds)])
`

In [ ]:
# we load the data from the pickle file
d = pd.read_pickle('data/fluorescence.p')[0.0]
args = [ d['t'], d['F'], d['Ferr'] ] # list containing x, y, yerr
# we define the exponential decay
def func(x, param):
    return param[0] * np.exp( - x / param[1] ) 
# we define the function that calculates the residuals
def leastsq_function(param, *args):
    return ( args[1] - func(args[0], param) ) / args[2] # residuals weighted by the error on y

In [ ]:
# we fit the data
res = least_squares(leastsq_function, x0=[3000,150], args=args, bounds=[(6,150),(3500,200)])
plt.errorbar(args[0],args[1],args[2],lw=0,marker='o',ms=1,elinewidth=1.,capsize=3,capthick=1.,
             color='k',alpha=0.3)
plt.plot(args[0],func(args[0],res['x']),lw=3,color='r')
plt.yscale('log')
plt.ylabel('$\ln F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

In [ ]:
res.x

In [ ]:
from IPython import display

table = r"""| Parameter     | Value         | Error |
| :---------------- |:-------------:| :----:|
| $F(0)$ (counts)   | %.2f          | ?     |
| $\tau_0$ (ns)     | %.2f          | ?     |"""

display.Markdown(table % tuple(res.x) )

In [ ]:
print('Is one of the convergence criteria satisfied?', res.success) 

#### Comparison between gaussian with $\sigma=1$ and $\mu=0$ and the histogram of the residuals at solution

In [ ]:
bins = np.linspace(res.fun.min(),res.fun.max()+1,100)
plt.plot(bins,np.exp(-bins*bins/2)/np.sqrt(2*np.pi),label='Gaussian',color='k')
plt.hist(res.fun, bins, lw=2, histtype='step',normed=True,label='residuals',color='r')
plt.xlabel('Residuals'); plt.ylabel('Probability Distribution')
plt.legend(); plt.show()

#### Estimating the Goodness of Fit and the Errors in the Fitting Parameters [(as in `optimize.curve_fit`)](https://github.com/scipy/scipy/blob/2526df72e5d4ca8bad6e2f4b3cbdfbc33e805865/scipy/optimize/minpack.py#L739)
We estimate the reduced chi squared, $\chi^2_R$, by summing over the square of the residuals at the solution and dividing by the degrees of freedom (number of points - number of parameters).<br>
Best-fit parameters uncertainties are estimated from the variance-covariance matrix.<br>
Further information [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) and [here](http://scipy-cookbook.readthedocs.io/items/FittingData.html).

In [ ]:
from scipy.linalg import svd
# copied from curve_fit 
def error_analysis(res):
    cost = 2 * res.cost  # res.cost is half sum of squares!
    s_sq = cost / (res.fun.size - res.x.size) # residual variance or reduced chi squared
    # Do Moore-Penrose inverse discarding zero singular values.
    _, s, VT = svd(res.jac, full_matrices=False)
    threshold = np.finfo(float).eps * max(res.jac.shape) * s[0]
    s = s[s > threshold]
    VT = VT[:s.size]
    pcov = np.dot(VT.T / s**2, VT) * s_sq # variance-covariance matrix
    return np.sqrt(np.diag(pcov)), s_sq # errors on the params are the diagonal elements of pcov

In [ ]:
table = r"""| Parameter     | Value         | Error |
| :---------------- |:-------------:| :----:|
| $F(0)$ (counts)   | %.0f          | %.0f  |
| $\tau_0$ (ns)     | %.1f          | %.1f  |"""

In [ ]:
p_and_err = np.repeat(res.x, 2)
p_and_err[1::2] = error_analysis(res)[0]
print('Reduced chi squared:',error_analysis(res)[1])
display.Markdown(table % tuple(p_and_err) )

####  Task: Fit the data for [quencher]=0 using `optimize.curve_fit`

In [ ]:
def func(x, F0, tau0):
    return F0 * np.exp( - x / tau0 ) 
popt,pcov = curve_fit(func, args[0], args[1], sigma=args[2], p0=[3000,150], bounds=[(6,150),(3500,200)])
plt.errorbar(args[0],args[1],args[2],lw=0,marker='o',ms=1,elinewidth=1.,capsize=3,capthick=1.,color='k',alpha=0.3)
plt.plot(args[0],func(args[0],popt[0],popt[1]),lw=3,color='r')
plt.yscale('log'); plt.ylabel('$\ln F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

In [ ]:
table = r"""| Parameter     | Value         | Error |
| :---------------- |:-------------:| :----:|
| $F(0)$ (counts)   | %.0f          | %.0f  |
| $\tau_0$ (ns)     | %.1f          | %.1f  |"""

p_and_err = np.repeat(popt, 2)
p_and_err[1::2] = np.sqrt(np.diag(pcov))
display.Markdown(table % tuple(p_and_err) )

#### Global Fit of the 5 curves with increasing [quencher]: $F(0)$, $\tau_0$, $k_q$ are global parameters while $m$ depends on [quencher]
* we create 3 `list`s (for x, y, and yerr) each containing 5 `NumPy` arrays (for the 5 curves)
* we create a `list` called $args$ containing the 3 `list`s ($allx$, $ally$, $allyerr$)

In [ ]:
data = pd.read_pickle('data/fluorescence.p')
allx = []; ally = []; allyerr = [] # three empty lists
for conc in data.columns:
    allx.append(data[conc]['t']); ally.append(data[conc]['F']); allyerr.append(data[conc]['Ferr'])
args = allx, ally, allyerr # args is a list of three lists containing 5 numpy arrays
param0 = [3000, 160, 0.04, 0] + [0.5]*4 # initial parameters
lbound = [0, 0, 0] + [0]*5              # lower bound
ubound = [4000, 200, 1, 1e-5] + [3]*4   # upper bound

In `leastsq_function`, we calculate the residuals for each curve which are assembled in a 1D `NumPy` array

In [ ]:
# we define the function for the intensity decay
def func(x, param):
    return param[0]*np.exp( - x/param[1] + param[3]*np.expm1(-param[2]*x) ) 
# we define the function that calculates the least squares
def leastsq_function(p, *args):
    residuals = np.empty(0)
    for i,m in enumerate(p[3:]):
        param = p[0], p[1], p[2], m
        r = ( args[1][i] - func(args[0][i], param) ) / args[2][i]
        residuals = np.append(residuals, r)
    return residuals

In [ ]:
res = least_squares(leastsq_function, x0=param0, args=args, bounds=[lbound,ubound])
c = plt.rcParams['axes.prop_cycle'].by_key()['color']
p = res['x']
for i,m in enumerate(p[3:]):
        param = p[0], p[1], p[2], m
        plt.errorbar(args[0][i],args[1][i],args[2][i],lw=0,marker='o',
                 ms=4,elinewidth=1.,capsize=3,capthick=1.,mfc='w',alpha=0.2)
        plt.plot(args[0][i],func(args[0][i],param),lw=3,color=c[i],label='{:1.1f}'.format(data.columns[i])+' mM')
plt.yscale('log'); plt.legend(title='[quencher]',labelspacing=.7)
plt.ylabel('$\ln F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

In [ ]:
table = r"""| Parameter     | Value         | Error     |
| :---------------- |:-------------:| :-------:|
| $F(0)$ (counts)   | %.0f          | %.0f     |
| $\tau_0$ (ns)     | %.1f          | %.1f     |
| $k_q$ (ns$^{-1}$) | %.4f          | %.4f     |
| $m_0$             | %.3f          | %.3f     |
| $m_1$             | %.3f          | %.3f     |
| $m_2$             | %.3f          | %.3f     |
| $m_3$             | %.3f          | %.3f     |
| $m_4$             | %.3f          | %.3f     |"""

In [ ]:
p_and_e = np.repeat(res['x'],2)
p_and_e[1::2] = error_analysis(res)[0]
print('Reduced chi squared:',error_analysis(res)[1])
display.Markdown(table % tuple(p_and_e) ) 

#### Task: Take the $\ln$ of each curve and perform a global fit to $\ln F(t) = \ln F(0) - t/\tau_0 + m [\exp(-k_q t) -1]$ using `least_square`.
* modify the `NumPy` arrays in args[1] by taking the `np.log`
* modify the `NumPy` arrays in args[2] according to the error propagation rule: error$(\ln F)$ = error($F$) / $F$
* define a new fitting function for the linearized equation 
* fitting and plotting can be carried out exactly as in the previous cell

In [ ]:
# we modify y and yerr in args
for i in range( data.columns.size ):
    args[2][i] = args[2][i]/args[1][i]
    args[1][i] = np.log(args[1][i]) 

# we define the function for the linearized intensity decay
def func(x, param):
    return param[0] - x/param[1] + param[3]*np.expm1(-param[2]*x)

In [ ]:
res = least_squares(leastsq_function, x0=param0, args=args, bounds=[lbound,ubound])
c = plt.rcParams['axes.prop_cycle'].by_key()['color']
p = res.x
for i,m in enumerate(p[3:]):
        param = p[0], p[1], p[2], m
        plt.errorbar(args[0][i],args[1][i],args[2][i],lw=0,marker='o',
                 ms=4,elinewidth=1.,capsize=3,capthick=1.,mfc='w',alpha=0.2)
        plt.plot(args[0][i],func(args[0][i],param),lw=3,color=c[i],label='{:1.1f}'.format(data.columns[i])+' mM')
plt.legend(title='[quencher]',labelspacing=.7)
plt.ylabel('$\ln F(t)$ / counts'); plt.xlabel('$t$ / ns'); plt.show()

In [ ]:
table = r"""| Parameter     | Value         | Error     |
| :---------------- |:-------------:| :-------:|
| $F(0)$ (counts)   | %.3f          | %.3f     |
| $\tau_0$ (ns)     | %.1f          | %.1f     |
| $k_q$ (ns$^{-1}$) | %.4f          | %.4f     |
| $m_0$             | %.3f          | %.3f     |
| $m_1$             | %.3f          | %.3f     |
| $m_2$             | %.3f          | %.3f     |
| $m_3$             | %.3f          | %.3f     |
| $m_4$             | %.3f          | %.3f     |"""

In [ ]:
p_and_e = np.repeat(res['x'],2)
p_and_e[1::2] = error_analysis(res)[0]
print('Reduced chi squared:',error_analysis(res)[1])
display.Markdown(table % tuple(p_and_e) ) 

In [ ]:
table = pd.DataFrame({'Value':res.x,'Error':error_analysis(res)[0]},
             index=['$F(0)$ (counts)','$\tau_0$ (ns)','$k_q$ (ns$^{-1}$)','$m_0$','$m_1$','$m_2$','$m_3$','$m_4$'],
            columns=['Value','Error'])

In [ ]:
table

In [ ]:
print(table.to_latex())